<a href="https://colab.research.google.com/github/helder-lourenco/IA-News/blob/main/Projeto_Imers%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerador de Agenda de Discussão de IA para Empresas

In [8]:
!pip install -q -U google-generativeai ipywidgets openpyxl requests
!pip install pandas==2.2.2 # Install pandas specifically after other packages

In [9]:
# Importar bibliotecas
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import json
import os
from datetime import datetime, timedelta


In [21]:
import google.generativeai as genai

In [24]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [25]:
# Configura a API Key do Google Gemini
# Importar o SDK do Google Gemini
import google.generativeai as genai
import os
from google.colab import userdata

# --- Configuração da API do Gemini ---
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Verifique se a API Key está configurada
try:
    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
    model = genai.GenerativeModel('gemini-pro')
    print("API do Gemini configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API do Gemini. Certifique-se de que GOOGLE_API_KEY está nas variáveis de ambiente do Colab. Erro: {e}")
    model = None # Define como None se a API não puder ser configurada



os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

API do Gemini configurada com sucesso!


In [26]:
# --- Variáveis Globais (para persistir dados entre as células) ---
df_projetos = None
agenda_gerada_texto = "" # Para armazenar a agenda gerada globalmente


In [28]:
# --- Seção 1: Dropdown de Segmentos da Empresa ---
segmentos_disponiveis = ['Selecione o Segmento',
    'Indústria Quimica', 'Tecnologia da Informação (TI)', 'Saúde', 'Finanças',
    'Varejo', 'Educação', 'Agronegócio', 'Serviços', 'Outros'
]

segmento_dropdown = widgets.Dropdown(
    options=segmentos_disponiveis,
    value=segmentos_disponiveis[0],
    description='Segmento da Empresa:',
    disabled=False,
)
display(segmento_dropdown)


Dropdown(description='Segmento da Empresa:', options=('Selecione o Segmento', 'Indústria Quimica', 'Tecnologia…

In [29]:
# --- Seção 2: Conectar com Planilha de Projetos ---
# Botões para conectar ou ignorar a planilha
conectar_planilha_button = widgets.Button(description="Conectar com Planilha de Projetos")
ignorar_planilha_button = widgets.Button(description="IGNORAR")

output_planilha = widgets.Output() # Para exibir mensagens ou o uploader

def on_conectar_planilha_clicked(b):
    with output_planilha:
        output_planilha.clear_output()
        print("Por favor, faça o upload da sua planilha (CSV ou XLSX).")
        from google.colab import files
        uploaded = files.upload() # Permite o upload de arquivos no Colab

        for fn in uploaded.keys():
            try:
                global df_projetos
                if fn.endswith('.csv'):
                    df_projetos = pd.read_csv(fn)
                elif fn.endswith('.xlsx'):
                    df_projetos = pd.read_excel(fn)
                else:
                    print("Formato de arquivo não suportado. Por favor, use .csv ou .xlsx.")
                    df_projetos = None
                    return

                # Validar colunas 'Projeto' e 'Status'
                if 'Projeto' in df_projetos.columns and 'Status' in df_projetos.columns:
                    print(f"Planilha '{fn}' carregada com sucesso!")
                    print("Primeiras 5 linhas:")
                    display(df_projetos.head())
                else:
                    print("A planilha deve conter as colunas 'Projeto' e 'Status'.")
                    df_projetos = None

            except Exception as e:
                print(f"Erro ao carregar o arquivo: {e}")
                df_projetos = None

def on_ignorar_planilha_clicked(b):
    with output_planilha:
        output_planilha.clear_output()
        print("Planilha de projetos ignorada. A agenda será baseada apenas no segmento.")
        global df_projetos
        df_projetos = None

conectar_planilha_button.on_click(on_conectar_planilha_clicked)
ignorar_planilha_button.on_click(on_ignorar_planilha_clicked)

display(widgets.HBox([conectar_planilha_button, ignorar_planilha_button]), output_planilha)


Output()

In [ ]:
# --- Função Auxiliar para Busca no Google (usando Google Search) ---
# Esta função simula uma busca no Google e retorna os snippets
# Em um ambiente de produção real, você usaria uma API de busca do Google mais robusta
# ou faria web scraping ético e controlado.

def perform_Google Search(query, num_results=5):
    try:
        # A ferramenta Google Search está disponível apenas em ambientes específicos (como no Gemini).
        # Para um Colab, você precisaria de uma API como Google Custom Search API ou fazer web scraping.
        # Por simplicidade e para simular a intenção, vamos usar um placeholder.
        # Substitua esta parte por uma chamada real à API de busca se tiver uma.
        # Ex: search_results = requests.get(f"https://www.googleapis.com/customsearch/v1?key=YOUR_CSE_API_KEY&cx=YOUR_CSE_ID&q={query}").json()

        # Placeholder para simular resultados de busca
        print(f"Simulando busca no Google para: '{query}'...")
        results = []
        if "inovação IA" in query.lower():
            results = [
                {"title": "IA Generativa em Saúde Revoluciona Diagnósticos", "link": "https://blog.ia-saude.com/generativa", "snippet": "Novas abordagens de IA criam tratamentos personalizados e detectam doenças em estágios iniciais."},
                {"title": "Robótica e IA Transformam Manufatura Inteligente", "link": "https://techmanufatura.com/robotica-ia", "snippet": "Sistemas autônomos com IA otimizam linhas de produção e controle de qualidade."},
                {"title": "IA na Previsão do Clima Extremo", "link": "https://climafuture.org/ia-previsao", "snippet": "Modelos de IA aprimoram a precisão na previsão de eventos climáticos severos."}
            ]
        elif "ferramentas IA" in query.lower():
            results = [
                {"title": "TensorFlow 3.0 Lançado com Novas Otimizações", "link": "https://www.tensorflow.org/blog/tf3", "snippet": "A última versão do TensorFlow traz melhorias de desempenho para GPUs e TPUs."},
                {"title": "PyTorch v2.5: Novidades para Desenvolvimento de LLMs", "link": "https://pytorch.org/blog/v2.5", "snippet": "PyTorch continua a inovar com ferramentas dedicadas para modelos de linguagem grandes."},
                {"title": "OpenAI Lança API para Criação de Agentes Autônomos", "link": "https://openai.com/agents-api", "snippet": "Nova API permite construir agentes de IA que interagem com o ambiente digital."}
            ]
        elif "notícias IA" in query.lower():
            results = [
                {"title": "Google Anuncia Nova Arquitetura de IA para Data Centers", "link": "https://news.google.com/google-ai-datacenter", "snippet": "Inovação visa reduzir o consumo de energia e aumentar a eficiência computacional."},
                {"title": "Debate Ético sobre IA em Armas Autônomas Ganha Força na ONU", "link": "https://un-ai-ethics.org/weapons", "snippet": "Líderes mundiais discutem a regulamentação do uso de IA em sistemas de defesa."},
                {"title": "Microsoft Adquire Startup de IA Conversacional por Bilhões", "link": "https://techcrunch.com/microsoft-ai-acquisition", "snippet": "Movimento estratégico da Microsoft para fortalecer sua posição no mercado de IA."}
            ]
        return results[:num_results] # Retorna o número de resultados solicitado
    except Exception as e:
        print(f"Erro na busca Google simulada: {e}")
        return []
